# Bokeh and Panda examples

In this jupyter notebook we will use the following libraries: bokeh, panda.

You can install those via *conda* with: `conda install bokeh panda`

In [1]:
import numpy as np

# Visualisation using BOKEH Library

In [3]:
#showing car's horsepower using custom made dataset

from bokeh.plotting import figure, output_notebook, show,save, ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Blues8 
import pandas as pd
#Read csv file
data= pd.read_csv("cars.csv")
car=data["Car"]
horsepower= data["Horsepower"]
output_notebook()
#Add plot
p= figure(y_range=car,  plot_width=800, plot_height=600, title="Car with Top HP", x_axis_label="Horsepower",
          tools="pan,box_select,zoom_in,zoom_out,save,reset")
#Render glyph
p.hbar(y=car, right=horsepower,left=0,height=0.4,color="orange",fill_alpha=0.5 )
show(p)

Loading BokehJS ...

In [4]:
# Adding colormaps and intractive image hovers using bokeh
#Read csv file
data= pd.read_csv("cars.csv")
#Create ColumnDataSource from data frame 
source = ColumnDataSource(data)
output_notebook()
#Car List
car_list = source.data["Car"].tolist()
#Add plot 
p= figure(y_range=car_list,  plot_width=800, plot_height=600, title="Car with Top HP", x_axis_label="Horsepower",
          tools="pan,box_select,zoom_in,zoom_out,save,reset")
#Render glyph/ For cmap use fill_color instead of color
p.hbar(y="Car", right="Horsepower",left=0,height=0.4,fill_color=factor_cmap("Car", palette=Blues8, factors=car_list),
       fill_alpha=0.9, source=source, legend="Car" )
#Add Legend
p.legend.orientation="vertical"
p.legend.location="top_right"
p.legend.label_text_font_size="10px"
#Ad  Tooltips:
hover =HoverTool()
hover.tooltips ="""
    <div>
        <h3>@Car</h3>
        <div><strong>Price:  </strong>@Price</div>
        <div><strong>HP:  </strong>@Horsepower</div>
        <div><img src="@Image" alt="" width="200" /></div>
    </div>
"""
p.add_tools(hover)
#Show results
show(p)
#Save file
save(p)


Loading BokehJS ...

'/var/folders/02/xz4ww2tn31vbt6727vwmp4zw0000gn/T/tmphlokajvs.html'

# Linking X Axes to create visualisation of 4 different teams' win or losses


In [5]:
# Isolate relevant data

team_stats = pd.read_csv('2017-18_teamBoxScore.csv', parse_dates=['gmDate'])
phi_gm_stats = (team_stats[(team_stats['teamAbbr'] == 'PHI') & 
                           (team_stats['seasTyp'] == 'Regular')]
                .loc[:, ['gmDate', 
                         'teamPTS', 
                         'teamTRB', 
                         'teamAST', 
                         'teamTO', 
                         'opptPTS',]]
                .sort_values('gmDate'))

# Add game number
phi_gm_stats['game_num'] = range(1, len(phi_gm_stats)+1)

# Derive a win_loss column
win_loss = []
for _, row in phi_gm_stats.iterrows():

    # If the 76ers score more points, it's a win
    if row['teamPTS'] > row['opptPTS']:
        win_loss.append('W')
    else:
        win_loss.append('L')

# Add the win_loss data to the DataFrame
phi_gm_stats['winLoss'] = win_loss

In [6]:
phi_gm_stats.head()

,gmDate,teamPTS,teamTRB,teamAST,teamTO,opptPTS,game_num,winLoss
10,2017-10-18,115,48,25,17,120,1,L
39,2017-10-20,92,47,20,17,102,2,L
52,2017-10-21,94,41,18,20,128,3,L
80,2017-10-23,97,49,25,21,86,4,W
113,2017-10-25,104,43,29,16,105,5,L


In [22]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Div
from bokeh.layouts import gridplot, column
output_notebook()

# Store the data in a ColumnDataSource
gm_stats_cds = ColumnDataSource(phi_gm_stats)

Loading BokehJS ...

In [23]:
win_loss_mapper = CategoricalColorMapper(factors = ['W', 'L'], 
                                         palette=['green', 'red'])

In [24]:
stat_names = {'Points': 'teamPTS',
              'Assists': 'teamAST',
              'Rebounds': 'teamTRB',
              'Turnovers': 'teamTO',}

# The figure for each stat will be held in this dict
stat_figs = {}

# For each stat in the dict
for stat_label, stat_col in stat_names.items():

    # Create a figure
    fig = figure(y_axis_label=stat_label, 
                 plot_height=200, plot_width=400,
                 x_range=(1, 10), tools=['xpan', 'reset', 'save'])

#     # Configure vbar
    fig.vbar(x='game_num', top=stat_col, source=gm_stats_cds, width=0.9, 
             color=dict(field='winLoss', transform=win_loss_mapper))

    # Add the figure to stat_figs dict
    stat_figs[stat_label] = fig

In [25]:
# Create layout
grid = gridplot([[stat_figs['Points'], stat_figs['Assists']], 
                [stat_figs['Rebounds'], stat_figs['Turnovers']]])

In [26]:
# Link together the x-axes
stat_figs['Points'].x_range = \
    stat_figs['Assists'].x_range = \
    stat_figs['Rebounds'].x_range = \
    stat_figs['Turnovers'].x_range

In [27]:
# Add a title for the entire visualization using Div
html = """<h3>Philadelphia 76ers Game Log</h3>
<b><i>2017-18 Regular Season</i>
<br>
</b><i>Wins in green, losses in red</i>
"""
sup_title = Div(text=html)

# Visualize
show(column(sup_title, grid))